In [27]:
import pymongo
import pandas as pd
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.job_data
job_collection = db.job_collection
population = db.population
universities = db.universities

In [118]:
job_df = pd.DataFrame(list(job_collection.find()))
population_df = pd.DataFrame(list(population.find()))
universities_df = pd.DataFrame(list(universities.find()))
job_df = job_df[job_df.title.str.contains("Scientist")]
grp_job = job_df.groupby(["state"]).count()
grp_job = grp_job.reset_index()
grp_job = grp_job.rename(columns={"_id": "job_count"})
grp_job = grp_job[["state", "job_count"]]
population_df.to_excel("population.xls", "population")
universities_df.to_excel("universities.xls", sheet_name="universities")
job_df.to_excel("jobs.xls", sheet_name="jobs")

universities_grp = universities_df.groupby("State").count()
universities_grp = universities_grp.reset_index()
universities_grp = universities_grp.rename(columns={"_id": "university_count", "State": "state"})
universities_grp = universities_grp[["state", "university_count"]]
population_df = population_df[["Population", "State"]]
population_df = population_df.rename(columns={"State": "state"})
population_df['Population'] = population_df['Population'].str.replace(',', '')
population_df['state'] = population_df['state'].apply(lambda x: x.strip())
population_df['Population'] = population_df['Population'].apply(lambda x: int(x))
population_df = population_df.set_index("state")
grp_job = grp_job.set_index("state")
universities_grp = universities_grp.set_index("state")
summary = grp_job.join(population_df).join(universities_grp).dropna()
summary.to_excel("summary.xls", sheet_name="summary")
